In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster, GroupedLayerControl

# TODOs:
- ~~TODO: find which neighborhood have more report in a ward~~
    - ~~Ward is easier and simpler~~
- ~~TODO: We should do proportions in the following graphs instead of raw numbers.~~
    - ~~With respect to number of all cases within given timeframe, or cases **with the same category (makes more sense)** within given timeframe~~
- ~~TODO: separate 2 different kind of graffiti.~~

# Data Cleaning and Processing

In [ ]:
# Data source: https://data.winnipeg.ca/Contact-Centre-311/311-Service-Request/4her-3th5
DF = pd.read_csv("./assets/311_Service_Request.csv")
DF.drop(["Zip Codes", "Location 1", "Wards", "Neighborhoods", "Electoral Ward 2018"], axis=1, inplace=True)
DF["Date"] = DF["Date"].astype('datetime64')
DF["Service Area and Request"] = DF["Service Request"].map(str) + ", " + DF["Service Area"]
DF.dropna(inplace=True)

DF["Neighbourhood"].replace("Logan-C.p.r.", "Logan-C.P.R.", inplace=True)

DF

## Services

In [ ]:
services = DF["Service Area and Request"].unique().tolist()
services.sort()
services

As shown above, the record currently have these services, divided into areas:

|Animal Services|Water|Street Maintenance|Sewer & Drainage|Garbage & Recycling|By Law Enforcement|Insect Control|Parks and Urban Foresty|
|---|---|---|---|---|---|---|---|
|Dog Complaint|Water Main Leak|Potholes|Sewer Backup|Missed Garbage Collection|Neighborhood Liveability Complaint|Mosquito Complaint|Graffiti|
|||Snow Removal - Sidewalks||Missed Recycling Collection||Tree Pest Caterpillar Complaint|Boulevard Mowing|
|||Graffiti||||||
|||Snow Removal - Roads||||||
|||Frozen Catch Basin||||||
|||Sanding||||||
|||Sidewalk Repairs||||||
|||Litter Container Complaint||||||

## Neighborhoods

### Overall distribution among service requests

In [ ]:
DF.groupby("Service Area and Request").size().sort_values().plot(kind="barh")

### Overall distribution among wards and neighbourhoods

In [ ]:
DF.groupby("Ward").size().sort_values().plot(kind="barh")

In [ ]:
DF.groupby("Neighbourhood").size().sort_values().plot(kind="barh", figsize=(9, 50))

Maybe these downtown hoods are closer to ~~waste station~~? Nope, currently there's only one waste landfill in Winnipeg.  
How about trashtrucks' depot?

### Overall Distribution among Requested services

In [ ]:
DF.groupby("Service Area and Request").size().sort_values().plot(kind="barh")

# Spatial

1. The number of events looks like related to population density. Examples are William Whyte and University.

In [ ]:
WINNIPEG = [49.88366050119829, -97.14581222292078]
m = folium.Map(location=WINNIPEG, zoom_start=12)


# By ward
data = DF.groupby("Ward").size()
ward_cp = folium.Choropleth(
    geo_data="assets/Electoral Ward.geojson",
    data=data,
    key_on="feature.properties.name",
    name="Ward"
).add_to(m)

for i in ward_cp.geojson.data['features']:
    i['properties']['events'] = str(data[i['properties']['name']])
folium.GeoJsonTooltip(['name', 'events']).add_to(ward_cp.geojson)

# By neighbourhood
data = DF.groupby("Neighbourhood").size()
hood_cp = folium.Choropleth(
    geo_data="assets/Neighbourhood.geojson",
    data=data,
    key_on="feature.properties.name",
    name="Neighbourhood"
).add_to(m)

for i in hood_cp.geojson.data['features']:
    if i['properties']['name'] != "Prairie Pointe":
        i['properties']['events'] = str(data[i['properties']['name']])
folium.GeoJsonTooltip(['name', 'events']).add_to(hood_cp.geojson)


# By case
def point_str_to_tuple(s: str) -> tuple:
    split = s.split()
    # return (float(split[1][1:]), float(split[2][:-1]))
    return (float(split[2][:-1]), float(split[1][1:]))

cases = MarkerCluster(name="Cases")
for i in DF.index:
    folium.Marker(
        location=point_str_to_tuple(DF['Point'][i]),
        # The popup takes 2 mins to load on 8-core RYZEN 5700X
        popup=f"<b>Date: </b>{DF['Date'][i]}<br><b>Service Area: </b>{DF['Service Area'][i]}<br><b>Service Request: </b>{DF['Service Request'][i]}"
    ).add_to(cases)
cases.add_to(m)


folium.LayerControl().add_to(m)
GroupedLayerControl( # example: https://github.com/chansooligans/folium/blob/81a04d3628b78b9538daadc3da81c9b1ee278692/examples/plugin-GroupedLayerControl.ipynb
    groups={"Division": [ward_cp, hood_cp]} # either ward or neibourhood, radio button choose one
).add_to(m)
m

In [ ]:
# Bar chart of Service Request for certain neibourhoods
def column_contains(column: str, neighbourhoods: list) -> str:
    ret = ""
    for neighbourhood in neighbourhoods:
        ret += f' or `{column}`.str.contains("{neighbourhood}", regex=False, na=False)'
    return ret[4:]

def top_n(n: int, group_by: str, ascending = False) -> list:
    return [key for key in DF.groupby(group_by).size().sort_values().nlargest(n).to_dict()] if not ascending else [key for key in DF.groupby(group_by).size().sort_values().nsmallest(n).to_dict()]


### Top Neibourhood from each ward

In [ ]:
wards = DF["Ward"].dropna().unique().tolist()

for ward in wards:
    print(f"For ward {ward}, the top neighbourhood is:")
    print(DF.query(column_contains("Ward", [ward])).groupby("Neighbourhood").size().sort_values(ascending=False)[0:1])

### Top request from each ward

In [ ]:
wards = DF["Ward"].dropna().unique().tolist()
n = 5

for ward in wards:
    print(f"For ward {ward}, the top {n} requests are:")
    print(DF.query(column_contains("Ward", [ward])).groupby("Service Area and Request").size().sort_values(ascending=False)[0:n])

### Top 10 Neighborhoods's request

In [ ]:
DF.query(column_contains("Neighbourhood", top_n(10, "Neighbourhood"))).groupby("Service Area and Request").size().sort_values().plot(kind="barh")


# Temporal

### Overall requests by Month

In [ ]:
DF.groupby(DF["Date"].dt.to_period("M")).size().plot(kind="bar", title="Overall requests by Month")

In [ ]:
(DF["Service Area and Request"] == services[3]).sum()

In [ ]:
def service_by_month(index: int, divideby=None):
    pr = pd.period_range(start=DF["Date"].min().strftime('%Y-%m'), end=DF["Date"].max().strftime('%Y-%m'), freq="M")
    pr = pd.Series(data = [0]*len(pr), index=pd.period_range(start="2021-01", end="2023-01", freq="M"))
    return (pr + DF.query(column_contains("Service Area and Request", [services[index]])).groupby(DF["Date"].dt.to_period("M")).size()).fillna(value=0).astype(int)

def plot_service_divide_by_month_percentage(index: int):
    service_by_month(index).divide(DF.groupby(DF["Date"].dt.to_period("M")).size()).plot(kind="bar", title=services[index])
    plt.show()

def plot_service_divide_by_overall_in_category(index: int):
    service_by_month(index).divide((DF["Service Area and Request"] == services[index]).sum()).plot(kind="bar", title=services[index])
    plt.show()

def plot_service_by_month(index: int):
    service_by_month(index).plot(kind="bar", title=services[index])
    plt.show()

In [ ]:
service_by_month(0).values

In [ ]:
# x = [str(period) for period in pd.period_range(start="2021-01", end="2023-01", freq="M")]
# y = []
# for i in range(len(services)):
#     y.append(service_by_month(i).values)
 
# # plot bars in stack manner
# plt.figure(figsize=(20, 20))
# plt.bar(x, y[0], color='r')
# for i in [0, 1, 2]:#range(1, len(services)):
#     plt.bar(x, y[i], bottom=y[i-1])
# # TODO: Completely messed up
# plt.legend(services)
# plt.show()

In [ ]:
x = [str(period) for period in pd.period_range(start="2021-01", end="2023-01", freq="M")]
y_list = []
select = [2, 10, 11, 14, 15]
for i in select: #range(len(services)):
    y_list.append(service_by_month(i).values)
y = np.vstack(y_list)

# plot
plt.figure(figsize=(20, 10))
plt.stackplot(x, y)

plt.legend([services[i] for i in select])
plt.show()

The data we have here doesn't have anything before the first day of 2021. How unfortunate. 

In [ ]:
plot_service_by_month(0)
plot_service_divide_by_overall_in_category(0)
plot_service_divide_by_month_percentage(0)

You don't mow grass in winter when there is no grass.

In [ ]:
plot_service_by_month(1)

In [ ]:
plot_service_by_month(2)

We had a lot of snow last winter, so it's understandable that catch basins are frozen and overwhelmed.

Further verify this by how much cumulative snow we got in the past years

In [ ]:
plot_service_by_month(3)

In [ ]:
plot_service_by_month(4)

There's a pattern.

People go out in a summer and a comfortable 25C summer night is perfect climate for graffiti. /s

Nobody want's to do graffiti in -30, right?

But that doesn't explain 2021-01 to 2021-02 why there are so many reports.

maybe useful, need fact check: Crime rate is higher in the winter, lower in the summer.

- What's happening in July 2022?
- What's wrong with Jan/Feb 2021? COVID policy, frustration? 
- COVID waves? Level of hospitalization vs infection(Omicron vs original COVID)

In [ ]:
plot_service_by_month(5)

This one matches Carson's hypothesis, garbage smell more in summer, but not in the winter.

In [ ]:
plot_service_by_month(6)

This is interesting.

As discussed with Carson, there maybe more reports on missed garbage collection in summer because it smells, and maybe less on winter because it doesn't smell(people stay inside all of the time, low temps is literally freezing your garbage so nothing could gong bad).

The overall trend is going up. Kind of sad.

But 2 of the peak are in 2022-03 and 2022-12. If you look one year back, from and before 2021-12, the reports are really stable.

Strike?

We need more data to make more out of this, like garbage amounts per month, labour situation, COVID policies, etc.

Residential or industrial, which made more garbage?

In [ ]:
plot_service_by_month(7)

It seems like there's a pattern? COVID is a thing to consider for sure.

In [ ]:
plot_service_by_month(8)

Fuck Mosquitos!

Good thing that mosquito are really sensitive to the temperature. 

But still why 2022 have almost 4x reports than 2021? Does it have something to do with the climate? Or COVID policies?

In [ ]:
plot_service_by_month(9)

That's a clear pattern, people are more active in the summer.

There was a dip in 2021-04, why? COVID restrictions?

In [ ]:
plot_service_by_month(10)

That's a clear pattern, but 2022 was much more dramatic than 2021.

In [ ]:
plot_service_by_month(11)

Compare with snow data

In [ ]:
plot_service_by_month(12)

Maybe the water level of Red River have something to do with this?

Probably need some civil engineering help.

In [ ]:
plot_service_by_month(13)

Hmmm that's interesting. As soon as winter come, the reports for sidewalk repairs plummets. I guess that's because nobody wants to walk outside of -30C. Also the snow covers everything, so the snow become the actual road surface.

But there were a lot of snow removal requests for sidewalks. We need to look at where exactly are these snow removal requests originated from. Are they from residential areas? People needs to walk in the neighborhood. Are they from industrial areas like Tuxedo?


In [ ]:
plot_service_by_month(14)

In [ ]:
plot_service_by_month(15)

Obviously we don't have snow in the summer, and that's good.

I do remember the Winter of 2021 there were a lot of snows. 

Almost perfect complement with sidewalk repairs

In [ ]:
plot_service_by_month(16)

What is this data, they do have a pattern that it's in the summer. There's a big gap between 2021-06 and 2022-06 data.

Note that this is complaint from the Winnipegger to 311, not the actual work dispatched. If we can have that data, that would explain something, like there were not enough people to work on this, and people are making duplicating requests.

Speaking of duplicate, does all of the data here contains duplications or not?

Because of COVIDin 2021, people are bored and looking for caterpillars.

In [ ]:
plot_service_by_month(17)

What exactly does this mean?

Maybe the water level of Red River have something to do with this?

I remember one of the summer the water level of red river was really high throughout the whole summer, the sidewalk along the river was completely submerged. That kinds of terrifies me because is that one of the consequences of global warming?

weather and temp related, more in the summer. If that's the case 2021-12 to 2022-03 are odd. Why is that?